### Indicadores de la Ciudad de México y a nivel alcaldía a partir de los resultados de la Encuesta Nacional de Seguridad Urbana
Recopila todos los datos de la encuesta (21-24) y se calculan los diferentes indicadores del Sistema del Instituto de Planeación de la CDMX.

In [2]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, readxl, writexl)

Cargando paquete requerido: pacman



### Descarga y lectura de datos

In [3]:
# Función para descomprimir los archivos ZIP y extraer todos los CSV
descomprimir_capas <- function(ruta_zip, carpeta_destino) {

options(timeout = 9000)

#Crear carpeta si no existe
if (!dir.exists(carpeta_destino)) {
  dir.create(carpeta_destino, recursive = TRUE)
}

#Descomprimir ZIP
  unzip(ruta_zip, exdir = carpeta_destino)
  
  archivos <- list.files(carpeta_destino, full.names = TRUE)
  print('Archivos extraídos del zip principal:')
  message(paste(archivos, collapse = '\n'))


  zip_files <- archivos[grepl('\\.zip$', archivos)]
  
  if (length(zip_files) > 0) {
    carpeta_2 <- file.path(carpeta_destino, 'Base de datos')
    dir.create(carpeta_2, showWarnings = FALSE)

    for (zip in zip_files) {
      tryCatch({
        unzip(zip, exdir = carpeta_2)
      }, error = function(e) {
        message('Error al descomprimir el archivo: ', zip, e$message)
    })
     
    }

    archivos_csv <- list.files(carpeta_2, pattern = 'ENSU_[0-9]{4}\\.csv$', full.names = TRUE)

    if (length(archivos_csv) > 0) {
      message('Archivos CSV encontrados:')
      message(paste(archivos_csv, collapse = '\n'))
      return(csv_files)
    }
  }
  
  return(NULL)
}

In [4]:
#Crear carpetas para guardar los microdatos y los resultados
dir.create('Microdatos', showWarnings = FALSE)

In [5]:
#Definición de url y años a iterar
url_base <- 'https://www.inegi.org.mx/contenidos/programas/ensu/microdatos/ensu_bd_'
years <- c(2021:2024)
data_list <- list()

In [6]:
#Función para determinar el trimestre desde el nombre del archivo
definir_trimestre <- function(nombre_file) {
    mes <- as.numeric(subst(nombre_file, 9, 10))
    trimestre <- case_when(
        mes %in% c(1:3) ~ 't1',
        mes %in% c(4:6) ~ 't2',
        mes %in% c(7:9) ~ 't3',
        mes %in% c(10:12) ~ 't4',
        TRUE ~ 'otro'
    )
    return(trimestre)
}


In [12]:
# Función para descomprimir archivos ZIP anidados
descomprimir_ensu <- function(ruta_zip, carpeta_destino) {
  # Aumentar timeout para descargas grandes
  options(timeout = 9000)
  
  # Crear carpeta si no existe
  if (!dir.exists(carpeta_destino)) {
    dir.create(carpeta_destino, recursive = TRUE)
  }
  
  # 1. Descomprimir archivo principal
  message('Descomprimiendo archivo principal: ', basename(ruta_zip))
  unzip(ruta_zip, exdir = carpeta_destino)
  
  # 2. Listar archivos extraídos (deberían ser ZIPs por trimestre)
  archivos <- list.files(carpeta_destino, full.names = TRUE)
  message('\nArchivos extraídos del zip principal:')
  message(paste(archivos, collapse = '\n'))
  
  # 3. Filtrar solo archivos ZIP (eliminar posibles archivos adicionales)
  zip_files <- archivos[grepl('\\.zip$', archivos, ignore.case = TRUE)]
  
  if (length(zip_files) == 0) {
    message('No se encontraron archivos ZIP internos')
    return(NULL)
  }
  
  # 4. Crear carpeta para bases de datos
  carpeta_bases <- file.path(carpeta_destino, 'Base de datos')
  dir.create(carpeta_bases, showWarnings = FALSE)
  
  # 5. Descomprimir cada ZIP interno y recolectar CSVs
  archivos_csv <- list()
  
  for (zip in zip_files) {
    tryCatch({
      message('\nDescomprimiendo archivo interno: ', basename(zip))
      
      # Descomprimir en subcarpeta con el nombre del trimestre
      trimestre <- gsub('ensu_bd_|_\\d{4}_csv\\.zip', '', basename(zip), ignore.case = TRUE)
      carpeta_trimestre <- file.path(carpeta_bases, trimestre)
      dir.create(carpeta_trimestre, showWarnings = FALSE)
      
      unzip(zip, exdir = carpeta_trimestre)
      
      # Encontrar archivos CSV en la subcarpeta
      csvs <- list.files(carpeta_trimestre, 
                         pattern = '^ENSU_CB_\\d{4}\\.csv$', 
                         full.names = TRUE,
                         ignore.case = TRUE)
      
      if (length(csvs) > 0) {
        message('Encontrados ', length(csvs), ' archivos CSV en ', basename(zip))
        archivos_csv <- c(archivos_csv, csvs)
      } else {
        message('No se encontraron archivos CSV en ', basename(zip))
      }
    }, error = function(e) {
      message('Error al descomprimir ', basename(zip), ': ', e$message)
    })
  }
  
  if (length(archivos_csv) == 0) {
    message('No se encontraron archivos CSV en ningún ZIP interno')
    return(NULL)
  }
  
  message('\nArchivos CSV encontrados en total: ', length(archivos_csv))
  return(unlist(archivos_csv))
}

# Función para determinar el trimestre desde el nombre del archivo
determinar_trimestre <- function(nombre_archivo) {
  # Extraer mes del nombre del archivo
  mes <- tolower(nombre_archivo)
  
  trimestre <- case_when(
    grepl('marzo', mes) ~ 'T1',
    grepl('junio', mes) ~ 'T2',
    grepl('septiembre', mes) ~ 'T3',
    grepl('diciembre', mes) ~ 'T4',
    TRUE ~ {
      # Alternativa: extraer mes del nombre del archivo CSV
      num_mes <- as.numeric(substr(basename(nombre_archivo), 9, 10))
      case_when(
        num_mes %in% 1:3 ~ 'T1',
        num_mes %in% 4:6 ~ 'T2',
        num_mes %in% 7:9 ~ 'T3',
        num_mes %in% 10:12 ~ 'T4',
        TRUE ~ 'Desconocido'
      )
    }
  )
  
  return(trimestre)
}


# Descargar y procesar datos por año
for (year in years) {
  tryCatch({
    # Configurar nombres de archivos y carpetas
    fin <- '_csv.zip'
    destfile <- paste0('Microdatos/ENSU_', year, fin)
    folder <- paste0('Microdatos/', year)
    
    # Crear carpeta para el año
    dir.create(folder, showWarnings = FALSE)
    
    # Descargar archivo
    download_url <- paste0(url_base, year, fin)
    message('\n[1/4] Descargando archivo ZIP desde: ', download_url)
    
    tryCatch({
      download.file(download_url, destfile, mode = 'wb')
      message('Descarga completada (', round(file.info(destfile)$size/1024/1024, 2), ' MB)')
    }, error = function(e) {
      message('Error en descarga: ', e$message)
      return(NULL)  # Saltar a la siguiente iteración si falla la descarga
    })
    
    # Verificar que el archivo ZIP existe y no está vacío
    if (!file.exists(destfile) || file.info(destfile)$size == 0) {
      message('El archivo ZIP no existe o está vacío')
      return(NULL)
    }
    
    message('\n[2/4] Descomprimiendo archivo ZIP...')
    archivos_csv <- descomprimir_ensu(destfile, folder)
    
    # Verificar extracción
    if (is.null(archivos_csv) || length(archivos_csv) == 0) {
      message('No se encontraron archivos CSV válidos')
      return(NULL)
    }
    message('Archivos CSV encontrados: ', length(archivos_csv))
    
    # Eliminar archivo ZIP después de descomprimir
    unlink(destfile)
    
    # Inicializar lista para el año
    data_list[[as.character(year)]] <- list()
    
    message('\n[3/4] Procesando archivos CSV...')
    # Leer cada archivo CSV
    for (file in archivos_csv) {
      message('\n Procesando: ', basename(file), ' ---')
      
      # Determinar trimestre basado en la ruta del archivo
      trimestre <- determinar_trimestre(dirname(file))
      message('Trimestre identificado: ', trimestre)
      
      # Verificar si ya tenemos datos para este trimestre
      if (!is.null(data_list[[as.character(year)]][[trimestre]])) {
        message('Ya existen datos para ', year, ' ', trimestre,)
      }
      
      # Leer el archivo CSV
      tryCatch({
        datos <- read_csv(
          file,
          locale = locale(encoding = 'ISO-8859-1'),
          show_col_types = FALSE,
          guess_max = 10000,
          progress = show_progress()) 
        
        # Limpiar nombres de columnas
        datos <- clean_names(datos)
        
        
        message('\n Archivo leído correctamente')
        message('Resumen del dataframe:')
        message('- Filas: ', nrow(datos))
        message('- Columnas: ', ncol(datos))
        
        # Combinar o guardar datos
        if (!is.null(data_list[[as.character(year)]][[trimestre]])) {
          # Combinar con datos existentes
          data_list[[as.character(year)]][[trimestre]] <- bind_rows(
            data_list[[as.character(year)]][[trimestre]],
            datos
          )
          message('Datos combinados en data_list[[', year, ']][[', trimestre, ']]')
        } else {
          # Guardar nuevos datos
          data_list[[as.character(year)]][[trimestre]] <- datos
          message('Datos guardados en data_list[[', year, ']][[', trimestre, ']]')
        }
        
      }, error = function(e) {
        message('Error al leer el archivo: ', e$message)
      })
    }
    
    message('\n[4/4] Resumen para ', year, ':')
    message('Trimestres procesados: ', paste(names(data_list[[as.character(year)]]), collapse = ', '))
    
  }, error = function(e) {
    message('Error general procesando año ', year, ': ', e$message)
  })
}


[1/4] Descargando archivo ZIP desde: https://www.inegi.org.mx/contenidos/programas/ensu/microdatos/ensu_bd_2021_csv.zip

Descarga completada (10.5 MB)


[2/4] Descomprimiendo archivo ZIP...

Descomprimiendo archivo principal: ENSU_2021_csv.zip


Archivos extraídos del zip principal:

Microdatos/2021/Base de datos
Microdatos/2021/ensu_bd_diciembre_2021_csv.zip
Microdatos/2021/ensu_bd_junio_2021_csv.zip
Microdatos/2021/ensu_bd_septiembre_2021_csv.zip


Descomprimiendo archivo interno: ensu_bd_diciembre_2021_csv.zip

Encontrados 1 archivos CSV en ensu_bd_diciembre_2021_csv.zip


Descomprimiendo archivo interno: ensu_bd_junio_2021_csv.zip

Encontrados 1 archivos CSV en ensu_bd_junio_2021_csv.zip


Descomprimiendo archivo interno: ensu_bd_septiembre_2021_csv.zip

Encontrados 1 archivos CSV en ensu_bd_septiembre_2021_csv.zip


Archivos CSV encontrados en total: 3

Archivos CSV encontrados: 3


[3/4] Procesando archivos CSV...


 Procesando: ENSU_CB_1221.csv ---

Warning message in eval_tidy

### Indicadores Visor CDMX

In [16]:
#Crear carpeta para resultados de Visor CDMX
dir.create('Visor_CDMX')

Warning message in dir.create("Visor_CDMX"):
"'Visor_CDMX' already exists"


Percepción de Inseguridad en la Ciudad de México

In [17]:
dir.create('Visor_CDMX/Resultados_Percepcion_Seguridad')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos para bog_efec
PobTot <- list() #Lista para vaciar valores de pob_total
PerSegu <- list() #Lista para vaciar los valores de percepción de seguridad

Warning message in dir.create("Visor_CDMX/Resultados_Percepcion_Seguridad"):
"'Visor_CDMX\Resultados_Percepcion_Seguridad' already exists"


In [18]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  PerSegu[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent',  
                          'r_def', 'bp1_1', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total a nivel CDMX
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular percepción de inseguridad a nivel CDMX
        PerSegu[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp1_1 == 2) %>%   # 2 = Inseguro
          group_by(fecha) %>%
          summarise(percepcion = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
PerSegu_consolidado <- consolidar_resultados(PerSegu)

# Consolidar resultados a nivel CDMX
resultados_CDMX <- pob_total_consolidado %>%
  left_join(PerSegu_consolidado, by = c("año", "trimestre", "fecha")) %>%
  mutate(
    percepcion = replace_na(percepcion, 0),
    porcentaje_percepcion = ifelse(pob_total > 0, percepcion * 100 / pob_total, NA_real_)
  ) %>%
  arrange(año, trimestre) %>%
  select(año, trimestre, fecha, pob_total, percepcion, porcentaje_percepcion)

# Mostrar estructura de los resultados
glimpse(resultados_CDMX)

# Guardar archivo consolidado
write_csv(resultados_CDMX, file = "Visor_CDMX/Resultados_Percepcion_Seguridad/Resultados_CDMX_histórico.csv")


Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 15
Columns: 6
$ año                   <chr> "2021", "2021", "2021", "2022", "2022", "2022", …
$ trimestre             <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1", …
$ fecha                 <chr> "2021-06-01", "2021-09-01", "2021-12-01", "2022-…
$ pob_total             <dbl> 7315973, 7387436, 7426187, 7428720, 7404584, 748…
$ percepcion            <dbl> 4974872, 4961769, 4950110, 5009793, 4716777, 452…
$ porcentaje_percepcion <dbl> 68.00014, 67.16497, 66.65749, 67.43817, 63.70077…


Personas en la Ciudad de México que creen que hospitales saturados y servicio ineficiente son problema

In [103]:
dir.create('Visor_CDMX/Resultados_Hospitales')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Hospitales <- list() #Lista para vaciar los valores de hospitales saturados

Warning message in dir.create("Visor_CDMX/Resultados_Hospitales"):
"'Visor_CDMX\Resultados_Hospitales' already exists"


In [104]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Hospitales[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent',  
                          'r_def', 'bp3_1_14', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total a nivel CDMX
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular percepción de inseguridad a nivel CDMX
        Hospitales[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp3_1_14 == 1) %>%   #1 = Hospitales saturados son un problema
          group_by(fecha) %>%
          summarise(hospitales = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
hospitales_consolidado <- consolidar_resultados(Hospitales)

# Consolidar resultados a nivel CDMX
resultados_CDMX <- pob_total_consolidado %>%
  left_join(hospitales_consolidado, by = c("año", "trimestre", "fecha")) %>%
  mutate(
    hospitales = replace_na(hospitales, 0),
    porcentaje_hospitales = ifelse(pob_total > 0, hospitales * 100 / pob_total, NA_real_)
  ) %>%
  arrange(año, trimestre) %>%
  select(año, trimestre, fecha, pob_total, hospitales, porcentaje_hospitales)

# Mostrar estructura de los resultados
glimpse(resultados_CDMX)

# Guardar archivo consolidado
write_csv(resultados_CDMX, file = "Visor_CDMX/Resultados_Hospitales/Resultados_CDMX_histórico.csv")


Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 15
Columns: 6
$ año                   <chr> "2021", "2021", "2021", "2022", "2022", "2022", …
$ trimestre             <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1", …
$ fecha                 <chr> "2021-06-01", "2021-09-01", "2021-12-01", "2022-…
$ pob_total             <dbl> 7315973, 7387436, 7426187, 7428720, 7404584, 748…
$ hospitales            <dbl> 2377910, 1947402, 2287319, 1895968, 1949331, 188…
$ porcentaje_hospitales <dbl> 32.50299, 26.36100, 30.80072, 25.52214, 26.32600…


Porcentaje de la población de la Ciudad de México que sufrieron algún episodio de extorsión (Semestral)

In [105]:
dir.create('Visor_CDMX/Resultados_Extorsion')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Extorsion <- list() #Lista para vaciar los valores de personas extorsionadas

Warning message in dir.create("Visor_CDMX/Resultados_Extorsion"):
"'Visor_CDMX\Resultados_Extorsion' already exists"


In [106]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Extorsion[[year]] <- list()
  
  # Iterar sobre los trimestres seleccionados
  for (trimestre in c("T2", "T4")) {
    # Obtener datos del trimestre actual
    datos_semestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_semestre) && is.data.frame(datos_semestre) && ncol(datos_semestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_semestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 
                          'r_def', 'bp1_6_6', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total a nivel CDMX
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular extorsión a nivel CDMX
        Extorsion[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp1_6_6 == 1) %>%   # 1 = Ha sufrido extorsión
          group_by(fecha) %>%
          summarise(extorsion = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_semestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, Trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
extorsion_consolidado <- consolidar_resultados(Extorsion)

# Consolidar resultados a nivel CDMX
resultados_CDMX <- pob_total_consolidado %>%
  left_join(extorsion_consolidado, by = c("año", "trimestre", "fecha")) %>%
  mutate(
    extorsion = replace_na(extorsion, 0),
    porcentaje_extorsion = ifelse(pob_total > 0, extorsion * 100 / pob_total, NA_real_)
  ) %>%
  arrange(año, trimestre) %>%
  select(año, trimestre, fecha, pob_total, extorsion, porcentaje_extorsion)

# Mostrar estructura de los resultados
glimpse(resultados_CDMX)

# Guardar archivo consolidado
write_csv(resultados_CDMX, file = "Visor_CDMX/Resultados_Extorsion/Resultados_CDMX_consolidado.csv")


Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros



Rows: 8
Columns: 6
$ año                  <chr> "2021", "2021", "2022", "2022", "2023", "2023", "…
$ trimestre            <chr> "T2", "T4", "T2", "T4", "T2", "T4", "T2", "T4"
$ fecha                <chr> "2021-06-01", "2021-12-01", "2022-06-01", "2022-1…
$ pob_total            <dbl> 7315973, 7426187, 7404584, 7474213, 7496019, 7573…
$ extorsion            <dbl> 1021733, 708036, 1091812, 945198, 811182, 1016903…
$ porcentaje_extorsion <dbl> 13.965784, 9.534314, 14.745082, 12.646121, 10.821…


In [107]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Extorsion[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent',  
                          'r_def', 'bp1_6_6', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total a nivel CDMX
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular percepción de inseguridad a nivel CDMX
        Extorsion[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp1_6_6 == 1) %>%   #1 = Ha sufrido extorsión 
          group_by(fecha) %>%
          summarise(extorsion = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
extorsion_consolidado <- consolidar_resultados(Extorsion)

# Consolidar resultados a nivel CDMX
resultados_CDMX <- pob_total_consolidado %>%
  left_join(extorsion_consolidado, by = c("año", "trimestre", "fecha")) %>%
  mutate(
    extorsion = replace_na(extorsion, 0),
    porcentaje_extorsion = ifelse(pob_total > 0, extorsion * 100 / pob_total, NA_real_)
  ) %>%
  arrange(año, trimestre) %>%
  select(año, trimestre, fecha, pob_total, extorsion, porcentaje_extorsion)

# Mostrar estructura de los resultados
glimpse(resultados_CDMX)

# Guardar archivo consolidado
write_csv(resultados_CDMX, file = "Visor_CDMX/Resultados_extorsion/Resultados_CDMX_histórico.csv")


Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Error procesando año 2021, trimestre T3: In argument: `bp1_6_6 == 1`.

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Error procesando año 2022, trimestre T1: In argument: `bp1_6_6 == 1`.

Error procesando año 2022, trimestre T3: In argument: `bp1_6_6 == 1`.

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Error procesando año 2023, trimestre T1: In argument: `bp1_6_6 == 1`.

Error procesando año 2023, trimestre T3: In argument: `bp1_6_6 == 1`.

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Error procesando año 2024, trimestre T1: In argument: `bp1_6_6 == 1`.

Error procesando año 2024, trimestre T3: In argument: `bp1_6_6 == 1`.



Rows: 15
Columns: 6
$ año                  <chr> "2021", "2021", "2021", "2022", "2022", "2022", "…
$ trimestre            <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1", "…
$ fecha                <chr> "2021-06-01", "2021-09-01", "2021-12-01", "2022-0…
$ pob_total            <dbl> 7315973, 7387436, 7426187, 7428720, 7404584, 7485…
$ extorsion            <dbl> 1021733, 0, 708036, 0, 1091812, 0, 945198, 0, 811…
$ porcentaje_extorsion <dbl> 13.965784, 0.000000, 9.534314, 0.000000, 14.74508…


ERROR: Error: Cannot open file for writing:
* 'D:/Escritorio/VSC/ENSU/Visor_CDMX/Resultados_Extorsion/Resultados_CDMX_histórico.csv'


### Indicadores Visor Alcaldías

In [ ]:
#Crear carpeta para resultados de Visor Alcaldías
dir.create('Visor_Alcaldías')

Warning message in dir.create("Visor_Alcaldías"):
"'Visor_Alcaldías' already exists"


Población que cree que el gobierno de su alcaldía es efectivo

In [ ]:
dir.create('Visor_Alcaldías/Resultados_Gob_Efectivo')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos para bog_efec
PobTot <- list() #Lista para vaciar valores de pob_total
GobEfectivo <- list() #Lista para vaciar los valores de gob_efect

Warning message in dir.create("Visor_Alcaldías/Resultados_Gob_Efectivo"):
"'Visor_Alcaldías\Resultados_Gob_Efectivo' already exists"


In [ ]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  GobEfectivo[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 'cve_mun', 
                          'nom_cd', 'r_def', 'bp3_2', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular gobierno efectivo
        GobEfectivo[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp3_2 %in% c(1, 2)) %>%  # 1 = Muy efectivo, 2 = Efectivo
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(efectivo = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
gob_efectivo_consolidado <- consolidar_resultados(GobEfectivo)

# Consolidar resultados por alcaldía
resultados_por_alcaldia <- pob_total_consolidado %>%
  left_join(gob_efectivo_consolidado, 
            by = c("año", "trimestre", "fecha", "cve_mun", "nom_cd")) %>%
  mutate(
    efectivo = replace_na(efectivo, 0),
    porcentaje_efectivo = ifelse(pob_total > 0, efectivo * 100 / pob_total, NA_real_)
  ) %>%
  # Agrupar por alcaldía en lugar de por año
  arrange(cve_mun, año, trimestre) %>%
  select(cve_mun, nom_cd, año, trimestre, fecha, pob_total, efectivo, porcentaje_efectivo)

# Mostrar estructura de los resultados
glimpse(resultados_por_alcaldia)

# Guardar archivo consolidado
write_csv(resultados_por_alcaldia, file =  "Visor_Alcaldías/Resultados_Gob_Efectivo/1.Resultados_por_alcaldia_consolidada.csv")

# Opcional: Guardar un archivo por cada alcaldía
resultados_por_alcaldia %>%
  group_by(cve_mun, nom_cd) %>%
  group_walk(~ write_csv(.x, paste0("Visor_Alcaldías/Resultados_Gob_Efectivo/", 
                                   gsub(" ", "_", tolower(.y$nom_cd)), 
                                   "_", .y$cve_mun, 
                                   ".csv")))

Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 240
Columns: 8
$ cve_mun             <chr> "002", "002", "002", "002", "002", "002", "002", "…
$ nom_cd              <chr> "Azcapotzalco", "Azcapotzalco", "Azcapotzalco", "A…
$ año                 <chr> "2021", "2021", "2021", "2022", "2022", "2022", "2…
$ trimestre           <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1", "T…
$ fecha               <chr> "2021-06-01", "2021-09-01", "2021-12-01", "2022-03…
$ pob_total           <dbl> 355725, 353462, 349332, 349047, 353365, 366971, 37…
$ efectivo            <dbl> 100542, 100488, 89839, 54691, 96131, 87843, 143476…
$ porcentaje_efectivo <dbl> 28.26397, 28.42965, 25.71737, 15.66866, 27.20445, …


Percepción de Inseguridad en la alcaldía

In [ ]:
dir.create('Visor_Alcaldías/Resultados_Percepcion_Seguridad')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos para bog_efec
PobTot <- list() #Lista para vaciar valores de pob_total
PerSegu <- list() #Lista para vaciar los valores de percepción de seguridad

In [ ]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  PerSegu[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 'cve_mun', 
                          'nom_cd', 'r_def', 'bp1_1', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular percepción de inseguridad
        PerSegu[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp1_1 == 2) %>%   #2 = Inseguro
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(percepcion = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
PerSegu_consolidado <- consolidar_resultados(PerSegu)

# Consolidar resultados por alcaldía
resultados_por_alcaldia <- pob_total_consolidado %>%
  left_join(PerSegu_consolidado, 
            by = c("año", "trimestre", "fecha", "cve_mun", "nom_cd")) %>%
  mutate(
    percepcion = replace_na(percepcion, 0),
    porcentaje_percepcion = ifelse(pob_total > 0, percepcion * 100 / pob_total, NA_real_)
  ) %>%
  # Agrupar por alcaldía en lugar de por año
  arrange(cve_mun, año, trimestre) %>%
  select(cve_mun, nom_cd, año, trimestre, fecha, pob_total, percepcion, porcentaje_percepcion)

# Mostrar estructura de los resultados
glimpse(resultados_por_alcaldia)

# Guardar archivo consolidado
write_csv(resultados_por_alcaldia, file =  "Visor_Alcaldías/Resultados_Percepcion_Seguridad/1.Resultados_por_alcaldia_consolidada.csv")

# Opcional: Guardar un archivo por cada alcaldía
resultados_por_alcaldia %>%
  group_by(cve_mun, nom_cd) %>%
  group_walk(~ write_csv(.x, paste0("Visor_Alcaldías/Resultados_Percepcion_Seguridad/", 
                                   gsub(" ", "_", tolower(.y$nom_cd)), 
                                   "_", .y$cve_mun, 
                                   ".csv")))


Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 240
Columns: 8
$ cve_mun               <chr> "002", "002", "002", "002", "002", "002", "002",…
$ nom_cd                <chr> "Azcapotzalco", "Azcapotzalco", "Azcapotzalco", …
$ año                   <chr> "2021", "2021", "2021", "2022", "2022", "2022", …
$ trimestre             <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1", …
$ fecha                 <chr> "2021-06-01", "2021-09-01", "2021-12-01", "2022-…
$ pob_total             <dbl> 355725, 353462, 349332, 349047, 353365, 366971, …
$ percepcion            <dbl> 273072, 249354, 270301, 289468, 254540, 269371, …
$ porcentaje_percepcion <dbl> 76.76492, 70.54620, 77.37654, 82.93095, 72.03317…


Porcentaje de la población que perciben inseguras las calles que frecuentan

In [ ]:
dir.create('Visor_Alcaldías/Resultados_Calles_Inseguras')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos para bog_efec
PobTot <- list() #Lista para vaciar valores de pob_total
Calles_inseg <- list() #Lista para vaciar los valores de percepción de seguridad

In [ ]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Calles_inseg[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 'cve_mun', 
                          'nom_cd', 'r_def', 'bp1_2_03', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular calles inseguras
        Calles_inseg[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp1_2_03 == 2) %>%   #2 = Inseguro en calles
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(calles_inseg = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
calles_inseg_consolidado <- consolidar_resultados(Calles_inseg)

# Consolidar resultados por alcaldía
resultados_por_alcaldia <- pob_total_consolidado %>%
  left_join(calles_inseg_consolidado, 
            by = c("año", "trimestre", "fecha", "cve_mun", "nom_cd")) %>%
  mutate(
    calles_inseg = replace_na(calles_inseg, 0),
    porcentaje_calles_inseg = ifelse(pob_total > 0, calles_inseg * 100 / pob_total, NA_real_)
  ) %>%
  # Agrupar por alcaldía en lugar de por año
  arrange(cve_mun, año, trimestre) %>%
  select(cve_mun, nom_cd, año, trimestre, fecha, pob_total, calles_inseg, porcentaje_calles_inseg)

# Mostrar estructura de los resultados
glimpse(resultados_por_alcaldia)

# Guardar archivo consolidado
write_csv(resultados_por_alcaldia, file =  "Visor_Alcaldías/Resultados_calles_inseguras/1.Resultados_por_alcaldia_consolidada.csv")

# Opcional: Guardar un archivo por cada alcaldía
resultados_por_alcaldia %>%
  group_by(cve_mun, nom_cd) %>%
  group_walk(~ write_csv(.x, paste0("Visor_Alcaldías/Resultados_calles_inseguras/", 
                                    gsub(" ", "_", tolower(.y$nom_cd)), 
                                    "_", .y$cve_mun, 
                                    ".csv")))

Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 240
Columns: 8
$ cve_mun                 <chr> "002", "002", "002", "002", "002", "002", "002…
$ nom_cd                  <chr> "Azcapotzalco", "Azcapotzalco", "Azcapotzalco"…
$ año                     <chr> "2021", "2021", "2021", "2022", "2022", "2022"…
$ trimestre               <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1"…
$ fecha                   <chr> "2021-06-01", "2021-09-01", "2021-12-01", "202…
$ pob_total               <dbl> 355725, 353462, 349332, 349047, 353365, 366971…
$ calles_inseg            <dbl> 225962, 221842, 248624, 263573, 214464, 220782…
$ porcentaje_calles_inseg <dbl> 63.52154, 62.76262, 71.17126, 75.51218, 60.691…


Porcentaje de la población en la alcaldía que sufrieron algún episodio de extorsión (Semestral)

In [ ]:
dir.create('Visor_Alcaldías/Resultados_Extorsion')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Extorsion <- list() #Lista para vaciar los valores de personas extorsionadas

In [ ]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Extorsion[[year]] <- list()
  
  # Iterar sobre los trimestres seleccionados
  for (trimestre in c("T2", "T4")) {
    # Obtener datos del trimestre actual
    datos_semestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_semestre) && is.data.frame(datos_semestre) && ncol(datos_semestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_semestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 'cve_mun', 
                          'nom_cd', 'r_def', 'bp1_6_6', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular calles inseguras
        Extorsion[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp1_6_6 == 1) %>%   # 1 = Ha sufrido extorsión
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(extorsion = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_semestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, Trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
extorsion_consolidado <- consolidar_resultados(Extorsion)

# Consolidar resultados por alcaldía
resultados_por_alcaldia <- pob_total_consolidado %>%
  left_join(extorsion_consolidado, 
            by = c("año", "trimestre", "fecha", "cve_mun", "nom_cd")) %>%
  mutate(
    extorsion = replace_na(extorsion, 0),
    porcentaje_extorsion = ifelse(pob_total > 0, extorsion * 100 / pob_total, NA_real_)
  ) %>%
  arrange(cve_mun, año, trimestre) %>%
  select(cve_mun, nom_cd, año, trimestre, fecha, pob_total, extorsion, porcentaje_extorsion)

# Mostrar estructura de los resultados
glimpse(resultados_por_alcaldia)

# Guardar archivo consolidado
write_csv(resultados_por_alcaldia, file = "Visor_Alcaldías/Resultados_Extorsion/1.Resultados_por_alcaldia_consolidada.csv")

# Opcional: Guardar un archivo por cada alcaldía
resultados_por_alcaldia %>%
  group_by(cve_mun, nom_cd) %>%
  group_walk(~ write_csv(.x, paste0("Visor_Alcaldías/Resultados_Extorsion/", 
                                    gsub(" ", "_", tolower(.y$nom_cd)), 
                                    "_", .y$cve_mun, 
                                    ".csv")))


Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros



Rows: 128
Columns: 8
$ cve_mun              <chr> "002", "002", "002", "002", "002", "002", "002", …
$ nom_cd               <chr> "Azcapotzalco", "Azcapotzalco", "Azcapotzalco", "…
$ año                  <chr> "2021", "2021", "2022", "2022", "2023", "2023", "…
$ trimestre            <chr> "T2", "T4", "T2", "T4", "T2", "T4", "T2", "T4", "…
$ fecha                <chr> "2021-06-01", "2021-12-01", "2022-06-01", "2022-1…
$ pob_total            <dbl> 355725, 349332, 353365, 371940, 367734, 369369, 3…
$ extorsion            <dbl> 65619, 12177, 28831, 35294, 49442, 56239, 48405, …
$ porcentaje_extorsion <dbl> 18.446553, 3.485796, 8.158986, 9.489165, 13.44504…



Porcentaje de personas que dejaron de caminar en la noche

In [ ]:
dir.create('Visor_Alcaldías/Resultados_Caminar_Noche')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Caminar_noche <- list() #Lista para vaciar los valores de personas extorsionadas

In [ ]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Caminar_noche[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 'cve_mun', 
                          'nom_cd', 'r_def', 'bp1_5_2', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular calles inseguras
        Caminar_noche[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp1_5_2 == 1) %>%   #1 = Ha dejado de caminar de noche
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(caminar_noche = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
caminar_noche_consolidado <- consolidar_resultados(Caminar_noche)

# Consolidar resultados por alcaldía
resultados_por_alcaldia <- pob_total_consolidado %>%
  left_join(caminar_noche_consolidado, 
            by = c("año", "trimestre", "fecha", "cve_mun", "nom_cd")) %>%
  mutate(
    caminar_noche = replace_na(caminar_noche, 0),
    porcentaje_caminar_noche = ifelse(pob_total > 0, caminar_noche * 100 / pob_total, NA_real_)
  ) %>%
  # Agrupar por alcaldía en lugar de por año
  arrange(cve_mun, año, trimestre) %>%
  select(cve_mun, nom_cd, año, trimestre, fecha, pob_total, caminar_noche, porcentaje_caminar_noche)

# Mostrar estructura de los resultados
glimpse(resultados_por_alcaldia)

# Guardar archivo consolidado
write_csv(resultados_por_alcaldia, file =  "Visor_Alcaldías/Resultados_Caminar_Noche/1.Resultados_por_alcaldia_consolidada.csv")

# Opcional: Guardar un archivo por cada alcaldía
resultados_por_alcaldia %>%
  group_by(cve_mun, nom_cd) %>%
  group_walk(~ write_csv(.x, paste0("Visor_Alcaldías/Resultados_Caminar_Noche/", 
                                    gsub(" ", "_", tolower(.y$nom_cd)), 
                                    "_", .y$cve_mun, 
                                    ".csv")))

Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 240
Columns: 8
$ cve_mun                  <chr> "002", "002", "002", "002", "002", "002", "00…
$ nom_cd                   <chr> "Azcapotzalco", "Azcapotzalco", "Azcapotzalco…
$ año                      <chr> "2021", "2021", "2021", "2022", "2022", "2022…
$ trimestre                <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1…
$ fecha                    <chr> "2021-06-01", "2021-09-01", "2021-12-01", "20…
$ pob_total                <dbl> 355725, 353462, 349332, 349047, 353365, 36697…
$ caminar_noche            <dbl> 206518, 184287, 211496, 231345, 232698, 24870…
$ porcentaje_caminar_noche <dbl> 58.05552, 52.13771, 60.54298, 66.27904, 65.85…


Porcentaje de la población que cree que los baches en calles y avenidas son un problema

In [ ]:
dir.create('Visor_Alcaldías/Resultados_Baches')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Baches <- list() #Lista para vaciar los valores de personas extorsionadas

In [ ]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Baches[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 'cve_mun', 
                          'nom_cd', 'r_def', 'bp3_1_10', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular calles inseguras
        Baches[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp3_1_10 == 1) %>%   #Baches en calles y avenidas son un problema
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(baches = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
baches_consolidado <- consolidar_resultados(Baches)

# Consolidar resultados por alcaldía
resultados_por_alcaldia <- pob_total_consolidado %>%
  left_join(baches_consolidado, 
            by = c("año", "trimestre", "fecha", "cve_mun", "nom_cd")) %>%
  mutate(
    baches = replace_na(baches, 0),
    porcentaje_baches = ifelse(pob_total > 0, baches * 100 / pob_total, NA_real_)
  ) %>%
  # Agrupar por alcaldía en lugar de por año
  arrange(cve_mun, año, trimestre) %>%
  select(cve_mun, nom_cd, año, trimestre, fecha, pob_total, baches, porcentaje_baches)

# Mostrar estructura de los resultados
glimpse(resultados_por_alcaldia)

# Guardar archivo consolidado
write_csv(resultados_por_alcaldia, file =  "Visor_Alcaldías/Resultados_Baches/1.Resultados_por_alcaldia_consolidada.csv")

# Opcional: Guardar un archivo por cada alcaldía
resultados_por_alcaldia %>%
  group_by(cve_mun, nom_cd) %>%
  group_walk(~ write_csv(.x, paste0("Visor_Alcaldías/Resultados_Baches/", 
                                    gsub(" ", "_", tolower(.y$nom_cd)), 
                                    "_", .y$cve_mun, 
                                    ".csv")))

Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 240
Columns: 8
$ cve_mun           <chr> "002", "002", "002", "002", "002", "002", "002", "00…
$ nom_cd            <chr> "Azcapotzalco", "Azcapotzalco", "Azcapotzalco", "Azc…
$ año               <chr> "2021", "2021", "2021", "2022", "2022", "2022", "202…
$ trimestre         <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1", "T2"…
$ fecha             <chr> "2021-06-01", "2021-09-01", "2021-12-01", "2022-03-0…
$ pob_total         <dbl> 355725, 353462, 349332, 349047, 353365, 366971, 3719…
$ baches            <dbl> 256659, 260427, 276687, 307063, 301615, 265808, 2918…
$ porcentaje_baches <dbl> 72.15096, 73.67892, 79.20460, 87.97182, 85.35509, 72…


Porcentaje de la población que cree que las fallas y fugas de agua potable son un problema

In [ ]:
dir.create('Visor_Alcaldías/Resultados_Fuga_Agua')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Fuga_Agua <- list() #Lista para vaciar los valores de personas extorsionadas

In [ ]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Fuga_Agua[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 'cve_mun', 
                          'nom_cd', 'r_def', 'bp3_1_01', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular calles inseguras
        Fuga_Agua[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp3_1_01 == 1) %>%   #1 = Fallas y fugas de agua son un problema
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(fuga_agua = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
fuga_agua_consolidado <- consolidar_resultados(Fuga_Agua)

# Consolidar resultados por alcaldía
resultados_por_alcaldia <- pob_total_consolidado %>%
  left_join(fuga_agua_consolidado, 
            by = c("año", "trimestre", "fecha", "cve_mun", "nom_cd")) %>%
  mutate(
    fuga_agua = replace_na(fuga_agua, 0),
    porcentaje_fuga_agua = ifelse(pob_total > 0, fuga_agua * 100 / pob_total, NA_real_)
  ) %>%
  # Agrupar por alcaldía en lugar de por año
  arrange(cve_mun, año, trimestre) %>%
  select(cve_mun, nom_cd, año, trimestre, fecha, pob_total, fuga_agua, porcentaje_fuga_agua)

# Mostrar estructura de los resultados
glimpse(resultados_por_alcaldia)

# Guardar archivo consolidado
write_csv(resultados_por_alcaldia, file =  "Visor_Alcaldías/Resultados_Fuga_Agua/1.Resultados_por_alcaldia_consolidada.csv")

# Opcional: Guardar un archivo por cada alcaldía
resultados_por_alcaldia %>%
  group_by(cve_mun, nom_cd) %>%
  group_walk(~ write_csv(.x, paste0("Visor_Alcaldías/Resultados_Fuga_Agua/", 
                                    gsub(" ", "_", tolower(.y$nom_cd)), 
                                    "_", .y$cve_mun, 
                                    ".csv")))

Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 240
Columns: 8
$ cve_mun              <chr> "002", "002", "002", "002", "002", "002", "002", …
$ nom_cd               <chr> "Azcapotzalco", "Azcapotzalco", "Azcapotzalco", "…
$ año                  <chr> "2021", "2021", "2021", "2022", "2022", "2022", "…
$ trimestre            <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1", "…
$ fecha                <chr> "2021-06-01", "2021-09-01", "2021-12-01", "2022-0…
$ pob_total            <dbl> 355725, 353462, 349332, 349047, 353365, 366971, 3…
$ fuga_agua            <dbl> 240862, 238546, 279139, 281021, 245081, 250219, 2…
$ porcentaje_fuga_agua <dbl> 67.71017, 67.48844, 79.90651, 80.51093, 69.35633,…


Porcentaje de la población que creen que los hospitales saturados y servicio ineficiente son un problema

In [ ]:
dir.create('Visor_Alcaldías/Resultados_Hospitales')

variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Hospitales <- list() #Lista para vaciar los valores de hospitales saturados

In [ ]:
# Iterar sobre cada año en data_list
for (year in names(data_list)) {
  # Inicializar sublistas para cada año
  variables[[year]] <- list()
  PobTot[[year]] <- list()
  Hospitales[[year]] <- list()
  
  # Iterar sobre cada trimestre del año
  for (trimestre in names(data_list[[year]])) {
    # Obtener datos del trimestre actual
    datos_trimestre <- data_list[[year]][[trimestre]]
    
    # Verificar que los datos no sean nulos y tengan columnas
    if (!is.null(datos_trimestre) && is.data.frame(datos_trimestre) && ncol(datos_trimestre) > 0) {
      tryCatch({
        # Procesamiento de datos
        variables[[year]][[trimestre]] <- datos_trimestre %>%
          clean_names() %>%
          select(any_of(c('edad', 'upm', 'viv_sel', 'cve_ent', 'cve_mun', 
                          'nom_cd', 'r_def', 'bp3_1_14', 'fac_sel'))) %>%
          filter(cve_ent == '09', edad >= 18) %>% 
          mutate(
            fecha = case_when(
              trimestre == "T1" ~ paste0(year, "-03-01"),
              trimestre == "T2" ~ paste0(year, "-06-01"),
              trimestre == "T3" ~ paste0(year, "-09-01"),
              trimestre == "T4" ~ paste0(year, "-12-01"),
              TRUE ~ paste0(year, "-01-01")
            )
          )
        
        # Calcular población total
        PobTot[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(pob_total = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        # Calcular calles inseguras
        Hospitales[[year]][[trimestre]] <- variables[[year]][[trimestre]] %>%
          filter(bp3_1_14 == 1) %>%   #1 = Hospitales saturados y servicio deficiente son un problema
          group_by(cve_mun, nom_cd, fecha) %>%
          summarise(hospitales = sum(fac_sel, na.rm = TRUE), .groups = 'drop')
        
        message(sprintf("Procesado: Año %s, Trimestre %s - %d registros", year, trimestre, nrow(datos_trimestre)))
        
      }, error = function(e) {
        message(sprintf("Error procesando año %s, trimestre %s: %s", year, trimestre, e$message))
      })
    } else {
      message(sprintf("Datos faltantes: Año %s, Trimestre %s", year, trimestre))
    }
  }
}

# Función para consolidar los resultados de una lista anidada
consolidar_resultados <- function(lista_anidada) {
  map_dfr(lista_anidada, function(año) {
    map_dfr(año, ~.x, .id = "trimestre")
  }, .id = "año") %>%
    arrange(año, trimestre)
}

# Consolidar todos los resultados
pob_total_consolidado <- consolidar_resultados(PobTot)
hospitales_consolidado <- consolidar_resultados(Hospitales)

# Consolidar resultados por alcaldía
resultados_por_alcaldia <- pob_total_consolidado %>%
  left_join(hospitales_consolidado, 
            by = c("año", "trimestre", "fecha", "cve_mun", "nom_cd")) %>%
  mutate(
    hospitales = replace_na(hospitales, 0),
    porcentaje_hospitales = ifelse(pob_total > 0, hospitales * 100 / pob_total, NA_real_)
  ) %>%
  # Agrupar por alcaldía en lugar de por año
  arrange(cve_mun, año, trimestre) %>%
  select(cve_mun, nom_cd, año, trimestre, fecha, pob_total, hospitales, porcentaje_hospitales)

# Mostrar estructura de los resultados
glimpse(resultados_por_alcaldia)

# Guardar archivo consolidado
write_csv(resultados_por_alcaldia, file =  "Visor_Alcaldías/Resultados_Hospitales/1.Resultados_por_alcaldia_consolidada.csv")

# Opcional: Guardar un archivo por cada alcaldía
resultados_por_alcaldia %>%
  group_by(cve_mun, nom_cd) %>%
  group_walk(~ write_csv(.x, paste0("Visor_Alcaldías/Resultados_Hospitales/", 
                                    gsub(" ", "_", tolower(.y$nom_cd)), 
                                    "_", .y$cve_mun, 
                                    ".csv")))

Procesado: Año 2021, Trimestre T4 - 23428 registros

Procesado: Año 2021, Trimestre T2 - 22411 registros

Procesado: Año 2021, Trimestre T3 - 23356 registros

Procesado: Año 2022, Trimestre T4 - 24402 registros

Procesado: Año 2022, Trimestre T2 - 23688 registros

Procesado: Año 2022, Trimestre T1 - 23577 registros

Procesado: Año 2022, Trimestre T3 - 23618 registros

Procesado: Año 2023, Trimestre T4 - 24064 registros

Procesado: Año 2023, Trimestre T2 - 24435 registros

Procesado: Año 2023, Trimestre T1 - 23778 registros

Procesado: Año 2023, Trimestre T3 - 24493 registros

Procesado: Año 2024, Trimestre T4 - 23451 registros

Procesado: Año 2024, Trimestre T2 - 24114 registros

Procesado: Año 2024, Trimestre T1 - 24022 registros

Procesado: Año 2024, Trimestre T3 - 24096 registros



Rows: 240
Columns: 8
$ cve_mun               <chr> "002", "002", "002", "002", "002", "002", "002",…
$ nom_cd                <chr> "Azcapotzalco", "Azcapotzalco", "Azcapotzalco", …
$ año                   <chr> "2021", "2021", "2021", "2022", "2022", "2022", …
$ trimestre             <chr> "T2", "T3", "T4", "T1", "T2", "T3", "T4", "T1", …
$ fecha                 <chr> "2021-06-01", "2021-09-01", "2021-12-01", "2022-…
$ pob_total             <dbl> 355725, 353462, 349332, 349047, 353365, 366971, …
$ hospitales            <dbl> 75895, 79915, 64775, 68479, 100667, 58477, 10361…
$ porcentaje_hospitales <dbl> 21.33530, 22.60922, 18.54253, 19.61885, 28.48811…
